# **LLM Routing wtih Apigee**

<table align="left">
    <td style="text-align: center">
        <a href="https://colab.research.google.com/github/ssvaidyanathan/apigee-samples/blob/main/llm-routing/llm_routing_v1.ipynb">
          <img src="images/colab-logo-32px.png" alt="Google Colaboratory logo\"><br> Open in Colab
        </a>
      </td>
      <td style="text-align: center">
        <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fssvaidyanathan%2Fapigee-samples%2Fmain%2Fllm-routing%2Fllm_routing_v1.ipynb">
          <img width="32px" src="images/colab-ent.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
        </a>
      </td>    
      <td style="text-align: center">
        <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/ssvaidyanathan/apigee-samples/main/llm-routing/llm_routing_v1.ipynb">
          <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
        </a>
      </td>
      <td style="text-align: center">
        <a href="https://github.com/ssvaidyanathan/apigee-samples/blob/main/llm-routing/llm_routing_v1.ipynb">
          <img src="images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
        </a>
      </td>
</table>
<br />
<br />
<br />

# Routing Sample

- This is a sample Apigee proxy to demonstrate the routing capabilities of Apigee across different LLM providers. In this sample we will use Google VertexAI and Anthropic as the LLM providers
- The framework will easily help onboarding other providers using configurations

![architecture](./images/arch.jpg)

# Benefits of Routing with Apigee:

* **Configuration Driven Routing**: All the routing logic are driven through configuration which makes onboarding very easy
* **Security**: Irrespective of the model and providers, Apigee will secure the endpoints and
* **Consistency**: Apigee can offer that layer of consistency to work with any LLM SDKs that are being used

## Setup

Use the following GCP CloudShell tutorial. Follow the instructions to deploy the sample.

[![Open in Cloud Shell](https://gstatic.com/cloudssh/images/open-btn.png)](https://ssh.cloud.google.com/cloudshell/open?cloudshell_git_repo=https://github.com/ssvaidyanathan/apigee-samples&cloudshell_git_branch=main&cloudshell_workspace=.&cloudshell_tutorial=llm-routing/docs/cloudshell-tutorial.md)

## Test Sample

### Install Dependencies

In [ ]:
!pip install google-cloud-aiplatform

### Authenticate your notebook environment (Colab only)
If you are running this notebook on Google Colab, run the following cell to authenticate your environment. This step is not required if you are using Vertex AI Workbench.

In [8]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

#### Set the Variables

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
APIGEE_HOST="[your-apigee-host-domain]" # @param {type:"string"}
API_KEY="[your-apikey]" # @param {type:"string"}

if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    raise ValueError("Please set your PROJECT_ID")
if not APIGEE_HOST or APIGEE_HOST == "[your-apigee-host-domain]":
    raise ValueError("Please set your APIGEE_HOST")
if not API_KEY or API_KEY == "[your-apikey]":
    raise ValueError("Please set your API_KEY")

API_ENDPOINT = "https://"+APIGEE_HOST+"/v1/samples/llm-routing"
PROMPT="Suggest name for a flower shop"

#### Execute the Vertex AI Gemini model

In [ ]:
# from langchain_google_vertexai import VertexAI

# # Initialize the Vertex AI LLM
# llm = VertexAI(
#       project=PROJECT_ID,
#       location="us-east1",
#       api_endpoint=API_ENDPOINT,
#       api_transport="rest",
#       streaming=False,
#       credentials=None,
#       additional_headers={"x-apikey": API_KEY},
#       model_name="gemini-1.5-flash-001")

# llm.invoke(PROMPT)

import vertexai
from vertexai.generative_models import GenerativeModel

vertexai.init(
      project=PROJECT_ID,
      location="us-east1",
      api_endpoint=API_ENDPOINT,
      api_transport="rest",
      credentials=None)

model = GenerativeModel("gemini-1.5-flash-001")
response = model.generate_content(PROMPT)
print(response.text)


#### Execute the Vertex AI Anthropic model

In [ ]:
from anthropic import AnthropicVertex

# Initialize the Anthropic LLM
client = AnthropicVertex(
        base_url=API_ENDPOINT+"/v1",
        project_id=PROJECT_ID,
        region="us-east5",
        default_headers={"x-apikey": API_KEY}
        )

message = client.messages.create(

    model="claude-3-5-sonnet-v2@20241022",
    max_tokens=256,
    messages=[
        {
            "role": "user",
            "content": PROMPT,
        }
    ],
)
print(message.content[0].text)